In [2]:
import ccxt
import pandas as pd 
import numpy as np
import datetime
import pandas as pd 
import numpy as np
import random
import datetime
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt 
import utilities as utils
import fractionalDiff as fd
import pickle
import scipy.stats as sps
from sys import exit
import torch 
import torch.nn as nn
import time

device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

In [3]:
print(ccxt.exchanges)

['aax', 'ascendex', 'bequant', 'bibox', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bw', 'bybit', 'bytetrade', 'cdax', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinflex', 'coinmate', 'coinone', 'coinspot', 'crex24', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'eqonex', 'exmo', 'flowbtc', 'fmfwio', 'ftx', 'ftxus', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'liquid', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3'

In [4]:
binanceXch = ccxt.binance()

In [6]:
btc_usdt_ohlcv = binanceXch.fetch_ohlcv('BTC/USDT','15m', limit = 300)
candles = pd.DataFrame(btc_usdt_ohlcv, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])

candles['date'] = candles['date'].apply(lambda x: datetime.datetime.fromtimestamp(x // 1000))

rollWindows = [4, 6, 12, 16, 18, 20, 22, 24, 30, 48]

for rollWindow in rollWindows:
    window_size = 4
    mean = candles['close']
    mean = mean.rolling(window=rollWindow).mean()
    candles[f'ma_{str(rollWindow).zfill(2)}'] = mean


candles = candles[window_size:]
candles = candles.reset_index()
candles

bitcoin = pd.DataFrame()
bitcoin['date'] = candles['date']
bitcoin['indicator'] = ((candles['close'] - candles['open']) >= 0)*2-1

candles.reset_index().drop(columns = 'index', inplace=True)
candles.drop(columns = 'index', inplace=True)
candles

candles.to_excel('./data/candles.xlsx', engine='openpyxl',index = False)
bitcoin.to_excel('./data/bitcoin.xlsx', engine='openpyxl',index = False)

inputs = candles
inputVariables = inputs.columns[1:]
logicOutputs = bitcoin
tickers = ['indicator']


differenceOrder = 0.8 
window = 13
lags = 50

transformedInputs = fd.FractionallyDifferentiateInputs(inputs, inputVariables, order = differenceOrder, thresholdVal = lags)[lags:].reset_index()
currentDate = transformedInputs.iloc[-1]['date']
startDateIdx = transformedInputs.index[transformedInputs.date == currentDate][0] - window - 2 
stopDateIdx = transformedInputs.index[transformedInputs.date == currentDate][0] - 2
startDate = transformedInputs.date[startDateIdx]
stopDate = transformedInputs.date[stopDateIdx]

inputsToScale = utils.FilterInputs(transformedInputs, startDate, stopDate)
logicOutputsShifted = logicOutputs[lags:]
x, y = utils.SelectAndScale(inputsToScale, logicOutputsShifted, startDate, stopDate, tickers[0], alreadyScaled = False)
x.shape, y.shape

finalModel = skl_lm.LogisticRegression(random_state=0, verbose = 0).fit(x, y)
finalModel.predict(x[-1].reshape((1,-1)))

array([1])

array([-1])

In [7]:
a = 2 * (inputs.loc[(inputs.date>=startDate) & (inputs.date<=stopDate), 'close'] - inputs.loc[(inputs.date>=startDate) & (inputs.date<=stopDate), 'open'] >= 0) - 1
a

630   -1
631    1
632   -1
633   -1
634   -1
635    1
636    1
637   -1
638    1
639   -1
640    1
641    1
642    1
643   -1
dtype: int32